In [1]:
import torch
import torch.backends.cudnn as cudnn
import torchvision
import cv2 
from matplotlib import pyplot as plt
import os
import numpy as np
import PIL
import random
from PIL import Image
import h5py
from typing import Generator
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "2"  # Set the GPU 1 to use
from core.data_loader import sem_generator,patch_generator, load_whole_image
from core.crop_image import scale_f_num_0to3,crop_image, make_image_crop
from core.utils import seed_everything

/mnt/ssd/hyun/fbi-net/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
seed_everything(0) # Seed 고정

In [7]:
# load whole image[size : (2048, 3072)] in "whole_images" 
data_path = "Samsung_SNU"
num_per_Ffolder = 16
# whole_images = load_whole_image(data_path,num_per_Ffolder)

In [8]:
gen = sem_generator(data_path)

In [9]:
def generate_top_bottom_crop_image(data_path : str,generator : Generator[str,str,np.array]):
    for img in generator:
        set_num, f_num, image_num = img[:3]
        save_dir = os.path.join(data_path+"_1536x3072",set_num)
        os.makedirs(save_dir,exist_ok=True)
        #print(save_dir)
        img = img[-1][256:-256,:]
        #plt.imshow(img,cmap='gray')
        file_name = f"{f_num}_{image_num}.png"
        file_path = os.path.join(save_dir,file_name)
        plt.imsave(file_path,img,cmap='gray')

In [10]:
#generate_top_bottom_crop_image(data_path,gen)

In [11]:
!ls Samsung_SNU_1536x3072/SET1/F16_1.png

Samsung_SNU_1536x3072/SET1/F16_1.png
